## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras.layers import GlobalAveragePooling2D

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:

classifier=Sequential()

#卷積組合
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(32, (3,3), activation='relu', padding='same', input_shape=(32,32,3)))
classifier.add(BatchNormalization())


#卷積組合
classifier.add(Convolution2D(64, (3,3), padding='same', activation='relu'))
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(128, (3,3), padding='same', activation='relu'))
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(128, (3,3), padding='same', activation='relu'))
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(128, (3,3), padding='same', activation='relu'))
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2,2), padding='same'))

#flatten
# classifier.add(Flatten())
classifier.add(GlobalAveragePooling2D())

#FC
classifier.add(Dense(100, activation='relu')) #output_dim=100,activation=relu
classifier.add(Dropout(0.25))
#輸出
classifier.add(Dense(10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)



Epoch 1/100
500/500 [==============================] - 15s 30ms/step - loss: 1.4736 - accuracy: 0.4632
Epoch 2/100
500/500 [==============================] - 15s 30ms/step - loss: 1.0784 - accuracy: 0.6146
Epoch 3/100
500/500 [==============================] - 15s 31ms/step - loss: 0.8940 - accuracy: 0.6841
Epoch 4/100
500/500 [==============================] - 15s 31ms/step - loss: 0.7651 - accuracy: 0.7334
Epoch 5/100
500/500 [==============================] - 15s 31ms/step - loss: 0.6722 - accuracy: 0.7655
Epoch 6/100
500/500 [==============================] - 16s 32ms/step - loss: 0.5894 - accuracy: 0.7946
Epoch 7/100
500/500 [==============================] - 16s 32ms/step - loss: 0.5185 - accuracy: 0.8193
Epoch 8/100
500/500 [==============================] - 16s 32ms/step - loss: 0.4544 - accuracy: 0.8446
Epoch 9/100
500/500 [==============================] - 16s 32ms/step - loss: 0.4029 - accuracy: 0.8607
Epoch 10/100
500/500 [==============================] - 16s 32ms/step - l

500/500 [==============================] - 16s 32ms/step - loss: 0.0240 - accuracy: 0.9916
Epoch 78/100
500/500 [==============================] - 16s 32ms/step - loss: 0.0268 - accuracy: 0.9907
Epoch 79/100
500/500 [==============================] - 16s 31ms/step - loss: 0.0330 - accuracy: 0.9886
Epoch 80/100
500/500 [==============================] - 16s 31ms/step - loss: 0.0220 - accuracy: 0.9926
Epoch 81/100
500/500 [==============================] - 16s 31ms/step - loss: 0.0234 - accuracy: 0.9920
Epoch 82/100
500/500 [==============================] - 16s 31ms/step - loss: 0.0263 - accuracy: 0.9914
Epoch 83/100
500/500 [==============================] - 16s 31ms/step - loss: 0.0301 - accuracy: 0.9902
Epoch 84/100
500/500 [==============================] - 16s 31ms/step - loss: 0.0262 - accuracy: 0.9914
Epoch 85/100
500/500 [==============================] - 16s 31ms/step - loss: 0.0249 - accuracy: 0.9920
Epoch 86/100
500/500 [==============================] - 16s 31ms/step - loss:

In [5]:
# 取得test set score
score = classifier.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.336971402168274
Test accuracy: 0.8044000267982483


## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[2.2918046e-03, 7.9584929e-08, 1.0067928e-01, 8.6512440e-01,
        2.6429692e-02, 1.9606675e-07, 2.5599305e-05, 2.4247768e-03,
        3.0239278e-03, 2.6867065e-07]], dtype=float32)